In [0]:
unprocessed_source_last_load_date = dbutils.notebook.run('./source',60)

In [0]:
unprocessed_source_last_load_date

In [0]:
if spark.catalog.tableExists('duebu_datamodelling.bronze.bronze_data'):
  bronze_df = spark.sql(f'select max(order_date) as bronze_last_load_date from duebu_datamodelling.bronze.bronze_data')

  processed_last_load_date = str(bronze_df.collect()[0]['bronze_last_load_date'])
else:
  pass

In [0]:
processed_last_load_date

In [0]:
# Enable schema evolution
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

if not spark.catalog.tableExists('duebu_datamodelling.bronze.bronze_data'):
    processed_last_load_date = '1900-01-01'
    spark.sql('CREATE TABLE IF NOT EXISTS duebu_datamodelling.bronze.bronze_data')
    spark.sql(f'''
    INSERT INTO duebu_datamodelling.bronze.bronze_data
    SELECT * FROM duebu_datamodelling.bronze.source_data 
    WHERE last_updated > '{processed_last_load_date}' 
    AND last_updated <= '{unprocessed_source_last_load_date}'
    ''')
else:
    spark.sql(f'''
    INSERT INTO duebu_datamodelling.bronze.bronze_data
    SELECT * FROM duebu_datamodelling.bronze.source_data 
    WHERE last_updated > '{processed_last_load_date}' 
    AND last_updated <= '{unprocessed_source_last_load_date}'
    ''')



In [0]:
%sql
select max(order_date) ,count(*),max(order_id)
from duebu_datamodelling.bronze.bronze_data

In [0]:
%sql
select count(*) from duebu_datamodelling.bronze.bronze_data